In [1]:
%%capture
import pandas as pd

## 루트 디렉토리 설정 

In [2]:
# 작업 디렉토리 설정
path = r'D:\2020 구매기획팀\3. 정기업무\7. 포상 대상자 분석\파일 새로만들기\템플릿 업그레이드'

In [3]:
# 실적 데이터
file_name = r'구매실적.xlsx'

df = pd.read_excel(path + '\\' + file_name , encoding='cp949')
df.head(1)

,팀,담당자,구분,구매금액,절감액,도전목표
0,가공원료팀,강태성,일반,2.682716e+09,4.577772e+07,9.503641e+07


## 구매금액, 절감액, 절감률 

In [4]:
if df['절감액'].item == 0:
    df['절감률'] = 0
else:
    df['절감률'] = df['절감액']/(df['구매금액'] + df['절감액'])

## 목표달성률  

### 원료 

+ 목표 > 0: 

<blockquote>목표 달성률 = 1+ {(절감액 - 도전목표) / abs(도전목표)}</blockquote>

+ 목표 < 0: 

<blockquote>목표 달성률 = {(절감액 - 도전목표) / abs(도전목표)}</blockquote>

### 설비

+ 설비/외자: 절감률 / 7%
+ 순환: 제외 

### 인테리어

+ 인테리어: 절감률 / 4.5%

In [5]:
df.loc[df['도전목표'] < 0, '목표 달성률'] = (df['절감액'] - df['도전목표']) / abs(df['도전목표'])
df.loc[df['도전목표'] == 0, '목표 달성률'] = df['목표 달성률'] = 0
df.loc[df['도전목표'] > 0, '목표 달성률'] = 1 + \
    ((df['절감액'] - df['도전목표']) / abs(df['도전목표']))
df.loc[df['구분'] == '설비', '목표 달성률'] = df['절감률'] / 0.07
df.loc[df['구분'] == '외자', '목표 달성률'] = df['절감률'] / 0.07 #인수인계는 5.5%였으나, 김재욱 차장님이 7% 라고 하심
df.loc[df['구분'] == '인테리어', '목표 달성률'] = df['절감률'] / 0.045

In [6]:
df['index'] = df.loc[(df['구분']!='순환')].groupby('담당자')['구매금액'].rank(method='first', ascending=False)
df_achieved_rate = df[['팀','담당자', '구분','구매금액','절감액','도전목표','절감률','목표 달성률', 'index']]
df_achieved_rate.loc[(df_achieved_rate['구분']!='순환') & (df_achieved_rate['index'] <2)]

,팀,담당자,구분,구매금액,절감액,도전목표,절감률,목표 달성률,index
0,가공원료팀,강태성,일반,2.682716e+09,4.577772e+07,9.503641e+07,0.016778,0.481686,1.0
1,해외전략구매팀,곽범준,일반,6.789901e+08,-5.756220e+07,5.739443e+08,-0.092629,-0.100292,1.0
2,신선식품팀,권성민,일반,2.212994e+09,-3.149663e+07,1.466790e+08,-0.014438,-0.214732,1.0
3,해외전략구매팀,권태훈,일반,1.617868e+09,3.276159e+07,5.287613e+07,0.019848,0.619591,1.0
4,포장재팀,김동환,일반,2.533150e+09,7.966318e+06,6.700177e+07,0.003135,0.118897,1.0
5,상품소싱팀,김상숙,일반,1.957876e+09,2.514288e+07,8.455880e+06,0.012679,2.973420,1.0
6,간접구매팀,김선아,일반,1.599092e+09,7.785581e+07,2.250000e+07,0.046427,3.460258,1.0
7,신선식품팀,김성훈,일반,2.281641e+09,-3.731666e+07,-1.065748e+08,-0.016627,0.000000,1.0
8,간접구매팀,김웅걸,일반,2.265649e+09,1.569158e+08,4.583636e+07,0.064773,3.423391,1.0
9,해외전략구매팀,김자영,일반,2.963367e+09,8.556732e+07,3.250100e+07,0.028065,2.632759,1.0


## 신규아이디어 제안 성과 분석

### 신규절감제안 절감률 
+ 월 구매금액 대비 신규 절감 아이디어 제안 금액의 절감률
+ 신규아이디어 제안서 상정주차 기준으로 포상 기준월 담당자별 절감액 산출

In [7]:
# 신규아이디어 절감액 데이터 import
file_name2 = r'아이디어절감액.xlsx'
df2 = pd.read_excel(path + '\\' + file_name2 , encoding='cp949')
df2.head(1)

,담당자,아이디어절감액
0,이목원,1.908487e+06


In [8]:
df_sum = df.groupby('담당자')['구매금액'].sum()
df_sum = pd.DataFrame(df_sum)
df_sum.head(1)

,구매금액
담당자,
강태성,2.682716e+09


In [9]:
df_merge = pd.merge(df_sum, df2, how='inner', on='담당자')
df_merge['제안절감률'] = df_merge['아이디어절감액'] / df_merge['구매금액']
df_idea = df_merge
df_idea.head(5)

,담당자,구매금액,아이디어절감액,제안절감률
0,김동환,2.533150e+09,5.654233e+05,0.000223
1,김선아,1.599092e+09,1.536000e+05,0.000096
2,김웅걸,2.265649e+09,2.038333e+06,0.000900
3,김종훈,4.503707e+09,3.061575e+06,0.000680
4,김태현,1.188133e+09,1.480796e+06,0.001246


### 신규절감제안 달성률
+ 신규 아이디어 제안의 당월 목표 대비 절감 달성률
+ 원가절감 Tracking 목표 및 절감액 

<blockquote>신규절감제안 달성률 = 당월 제안절감액 / 당월 제안목표</blockquote>

In [10]:
# 원가절감 Tracking 데이터 import
file_name3 = r'원가절감Tracking.xlsx'
df3 = pd.read_excel(path + '\\' + file_name3 , encoding='cp949')
df3.head(1)

,담당자,제안목표,제안절감액
0,이종원,179347525.0,1.689757e+08


In [11]:
df3['제안달성률'] = df3['제안절감액'] / df3['제안목표']
df_idea_achieved = df3
df_idea_achieved.head(5)

,담당자,제안목표,제안절감액,제안달성률
0,이종원,1.793475e+08,1.689757e+08,0.942169
1,조진규,1.868229e+08,1.849610e+08,0.990034
2,이우진,1.604770e+08,1.125956e+08,0.701631
3,이승주,1.038791e+09,3.999570e+07,0.038502
4,김자영,6.878958e+07,1.981937e+07,0.288116


## 상품개발 성과 분석

### 상품개발 목표 대비 달성률 & 3개월 매출 평균  

+ 상품개발 달성률: 신규 상품개발의 당월 목표 대비 매출 달성률
<blockquote> 상품개발 달성률 = 당월 매출/ 상품개발목표 </blockquote>

+ 상품개발 매출액 평균: 최근 3개월 매출 평균금액 (단, 1월, 2월은 해당 월 까지의 누계 평균 반영)

In [12]:
# 원가절감 Tracking 데이터 import
file_name4 = r'상품개발실적.xlsx'
df4 = pd.read_excel(path + '\\' + file_name4 , encoding='cp949')
df4.head(1)

,담당자,상품개발목표,당월매출,3개월매출평균
0,김종훈,210000000.0,309030800,292726331.0


In [13]:
def divide_by_zero(a, b):
    if b.item == 0:
        return 0.0
    elif a.item == 0:
        return 0.0
    else:
        return a / b

df4['매출목표달성률'] = divide_by_zero(df4['당월매출'], df4['상품개발목표'])
df4

,담당자,상품개발목표,당월매출,3개월매출평균,매출목표달성률
0,김종훈,2.100000e+08,309030800,2.927263e+08,1.471575
1,조환,1.000000e+08,240284800,2.760147e+08,2.402848
2,여슬기한,3.420000e+07,0,3.000000e+04,0.000000
3,이철영,2.500000e+07,51884995,4.760619e+07,2.075400
4,권태훈,0.000000e+00,9800000,9.050000e+06,inf
5,김성훈,5.000000e+07,286466400,3.031278e+08,5.729328
6,윤해근,1.900000e+08,93402090,2.125926e+08,0.491590
7,이승주,4.540000e+08,34145000,1.758901e+08,0.075209
8,이종원,0.000000e+00,93639000,9.363900e+07,inf
9,김자영,5.000000e+07,3108450,1.908592e+07,0.062169
